# Figure 3: Phase Diagram of Helicity Rotation Behaviour

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib
import numpy as np
from colorsys import hls_to_rgb
import os
from PIL import Image

import sys
sys.path.append('../..')
from Utils.PhaseDiagramFunctions import *
from Utils.FiguresFunctions import *


plt.style.use('Style.mplstyle')


cc_directory = '../../CollectiveCoordinates/CollectiveCoordinateIntegration/CollectiveCoordinateDataSupplied'
mumax_directory = '../../Micromagnetics/SimulationsAndAnalysis/MicromagneticsDataSupplied'
R_directory = cc_directory + '/R/'
eta_directory = cc_directory + '/eta/'
E0_values, omega_values = get_E0_and_omega_arrays(R_directory)
helicity_criterion_array = np.load(cc_directory + '/PhaseDiagramData/HelicityCriterion.npy')
mean_winding_number = np.load(cc_directory + '/PhaseDiagramData/MeanWindingNumber.npy')

# Obtain parameters such as magnetic field, fit parameters
# In this case, we just need optimalSkyrmionRadius
Bz, dw_width, E_exchangeFitParams, E_magnetic_integralFitParams, E_electric_integralFitParams, alpha, \
    F_RexFitParams, Gamma11FitParams, Gamma22FitParams, G12FitParams, Xi1FitParams, Xi2FitParams, \
    Xi3FitParams, Xi4FitParams, optimalSkyrmionRadius = read_parameters(cc_directory)


def plot_coordinates(point_idx, ax_R, ax_eta, colour):
    
    """
    Plot radius and helicity over time (for bottom row).
    """

    E0 = E0_points[point_idx]
    omega = omega_points[point_idx]

    radii_mumax, helicities_mumax, radii_cc, helicities_cc = get_coordinate_plot_data(cc_directory, mumax_directory, E0, omega)
    times = np.load(cc_directory + '/General/times.npy')

    # Wrap to range [-pi, pi]
    for i in range(len(helicities_cc)):
        while helicities_cc[i] > np.pi:
            helicities_cc[i] -= 2*np.pi            
        while helicities_cc[i] < -np.pi:
            helicities_cc[i] += 2*np.pi

    # Take where helicity is closest to zero, but time is greater than 500 and less than 1000 - (time period of E_z)
    start_idx_initial = np.argmin(np.abs((helicities_cc[np.where((times > times[len(times)//2]) & (times < np.max(times)))]) % (2*np.pi)))
    time_start = times[start_idx_initial] + times[len(times)//2]

    indices_per_time_interval = int(np.round(1/(times[1]-times[0])))
    start_idx = int(np.round(time_start*indices_per_time_interval))
    end_idx = int(np.round((time_start + 2*np.pi/omega)*indices_per_time_interval)) + 1

    # Plot times from 0 to have the same starting point for all values of E0 and omega
    ax_R.plot(times[:(end_idx - start_idx)], radii_cc[start_idx:end_idx], '--', color=colour, label='Collective Coordinates')
    ax_R.plot(times[:(end_idx - start_idx)], radii_mumax[start_idx:end_idx], '-', color=colour, label='Micromagnetics')

    # The unwrap is to prevent the jump at the start or end
    ax_eta.plot(times[:(end_idx - start_idx)], np.unwrap(helicities_cc[start_idx:end_idx]) / np.pi, '--', color=colour, label='Collective Coordinates')
    ax_eta.plot(times[:(end_idx - start_idx)], np.unwrap(helicities_mumax[start_idx:end_idx]) / np.pi, '-', color=colour, label='Micromagnetics')

    time_tick_idcs = [int(np.round((i/4)*(end_idx-start_idx))) for i in range(5)]
    time_ticks = [times[i] for i in time_tick_idcs]

    ax_R.set_xticks(time_ticks)
    ax_R.set_xticklabels([r'$0$', r'$T/4$', r'$T/2$', r'$3T/4$', r'$T$']) 
    ax_eta.set_xticks(time_ticks)
    ax_eta.set_xticklabels([r'$0$', r'$T/4$', r'$T/2$', r'$3T/4$', r'$T$'])
    
    
def render_preimages(E0, omega):
    
    """
    Read in collective coordinate integration data and render preimage in POV-Ray.
    """
    
    R_sol = np.load(cc_directory + '/R/E0%.2fomega%.2f.npy' %(E0, omega))
    eta_sol = np.load(cc_directory + '/eta/E0%.2fomega%.2f.npy' %(E0, omega))
    times = np.load(cc_directory + '/General/times.npy')

    # Start at a the point above t = 500, with helicity closest to zero
    start_idx = get_start_idx(eta_sol, times, omega)
    time_start = times[start_idx] + times[len(times)//2]

    # List of of preimages to plot of form [[azimuth1, altitude1], [azimuth2, altitude2], ...]
    az_alt_list = [[0, np.pi/2], [np.pi, np.pi/2]]

    # Number of points on a time mesh of dimensionless time length 1
    indices_per_time_interval = int(np.round(1/(times[1]-times[0])))

    start_idx = int(np.round(time_start*indices_per_time_interval))
    end_idx = int(np.round((time_start + 2*np.pi/omega)*indices_per_time_interval)) + 1

    # For setting how wide and tall the preimages are
    radius_scale = 3
    time_scale = 5

    sphere_sweep_string = ''

    # Loop through the preimages
    for preimage_idx in range(len(az_alt_list)):

        az = az_alt_list[preimage_idx][0]
        alt = az_alt_list[preimage_idx][1]

        s = 1.
        l = 0.5*np.cos(alt) + 0.5
        h = az / (2*np.pi)

        r, g, b = hls_to_rgb(h, l, s)

        sphere_sweep_string = sphere_sweep_string + 'sphere_sweep {\n\
            linear_spline ' + str(end_idx - start_idx) + ',\n'

        x_preimage_values = R_sol * (1 - alt / np.pi) * np.cos(az - eta_sol)
        y_preimage_values = R_sol * (1 - alt / np.pi) * np.sin(az - eta_sol)


        # For each point in the collective coordinate integration for the period shown, add point to sphere_sweep
        for i in range(start_idx, end_idx):

            x_coordinate = radius_scale * x_preimage_values[i] / R_sol.max()
            y_coordinate = radius_scale * y_preimage_values[i] / R_sol.max()
            z_coordinate = time_scale * ((times[i] - times[start_idx]) / (times[end_idx] - times[start_idx]))

            sphere_sweep_string = sphere_sweep_string + '<' + str(x_coordinate) + ',' + str(z_coordinate) + \
            ',' + str(y_coordinate) + '>, sphere_radius\n'

        sphere_sweep_string = sphere_sweep_string + 'pigment { color <' + str(r) + ',' + str(g) + ',' + str(b) + '> }\n\
            finish {\n\
                phong 1\n\
            }\n\
            no_shadow\n\
        }\n\n'

    # Write out the POV-Ray script (created using template Template.pov)
    with open('Render.pov', 'w') as outfile:
        with open('Template.pov') as infile:
            template = infile.read()
            template = template.replace('{{ L }}', str(time_scale))
            template = template.replace('{{ NoSpheres }}', str(end_idx - start_idx))
            template = template.replace('{{ Sphere Sweep }}', sphere_sweep_string)
            outfile.write(template)
            
    # Render the POV-Ray script and rename it to match E0, omega
    os.system('povray RenderOptions.ini &>/dev/null')
    os.rename('Rendered.png', 'E0%.2fomega%.2f.png' %(E0, omega))
    


fig = plt.figure(figsize=(7, 12))
gs = fig.add_gridspec(3, 2)

# Cut off if E0 is above 2 for some points
max_E0_to_show = 1.5

ax_phase_diagram = fig.add_subplot(gs[0, :])

# Plot phase diagram
ax_phase_diagram.imshow(helicity_criterion_array, extent=[omega_values[0], omega_values[-1], E0_values[0], E0_values[-1]], origin='lower', interpolation='nearest')
ax_phase_diagram.set_ylabel(r'$E_0$', fontsize=20)
ax_phase_diagram.set_xlabel(r'$\omega$', fontsize=20)

# Add label at resonance frequency
resonance_colour = '#FF2B2B'
ax_phase_diagram.text(1.13, 1.37, r'$\omega_{\mathrm{res}}$', fontsize=25, color=resonance_colour)
ax_phase_diagram.axvline(1., lw=4, linestyle='--', color=resonance_colour)

# Insets of preimages for the two points shown
E0_points = [0.25, 0.20]
omega_points = [0.72, 1.18]
x_loc_points = [0.1, 0.3]
y_loc_points = [0.4, 0.52]
size = 0.5

grey_lightness = 0.8
prop = dict(arrowstyle="-|>,head_width=0.4,head_length=0.8",
            lw=3, color=[grey_lightness, grey_lightness, grey_lightness])

# Get aspect ratios to allow plots to be aligned whilst preserving equal aspect ratio, but adjusting y-limits
aspect_ratios = np.zeros(len(E0_points))

for i in range(len(aspect_ratios)):
    max_xvalue, max_yvalue = get_evolution_plot_axis_limits(cc_directory, mumax_directory, E0_points[i], omega_points[i])
    aspect_ratios[i] = max_xvalue / max_yvalue

min_aspect_ratio = aspect_ratios.min()

label_texts = {0: r'b', 1: r'c'}

# Deal with colours (make consistent with phase diagram)
cmap = matplotlib.cm.get_cmap('PiYG')
norm = matplotlib.colors.Normalize(vmin=-1, vmax=1)
plot_colours = {0: cmap(norm(1)), 1: cmap(norm(-1))}

for i in range(len(E0_points)):
    
    # Create preimage PNGs to be used in the insets
    render_preimages(E0_points[i], omega_points[i])

    # Add arrows for insets pointing to corresponding point in phase diagram
    ax_phase_diagram.annotate('', xy=(omega_points[i], E0_points[i]), xytext=((x_loc_points[i] + size/3)*np.max(omega_values), (y_loc_points[i] + size/2)*max_E0_to_show), 
                arrowprops=prop)

    # Add the preimages PNGs as insets
    ax_inset = ax_phase_diagram.inset_axes([x_loc_points[i], y_loc_points[i], size, size])
    ax_inset.text(0, 0.8, label_texts[i], transform=ax_inset.transAxes, size=18)
    img = np.asarray(Image.open('E0%.2fomega%.2f.png' %(E0_points[i], omega_points[i])))
    ax_inset.imshow(img)
    ax_inset.set_facecolor((0, 0, 0, 1))  # Make transparent
    ax_inset.axis('off')

    ax_phase_diagram.plot(omega_points[i], E0_points[i])

    # Plot the Rcos(eta)-Rsin(eta) plots
    ax_limitcycle = fig.add_subplot(gs[1, i])
    plot_evolution(cc_directory, mumax_directory, ax_limitcycle, E0_points[i], omega_points[i], min_aspect_ratio, plot_colours[i])

    if i == 0:
        dashed_line = mlines.Line2D([], [], linestyle='--', color='black', label='Collective Coordinates')
        full_line = mlines.Line2D([], [], linestyle='-', color='black', label='Micromagnetics')
        ax_limitcycle.legend(handles=[full_line, dashed_line], loc='center')

ax_phase_diagram.set_ylim(0, max_E0_to_show)


# Deal with legend
cmap = matplotlib.cm.get_cmap('PiYG')
norm = matplotlib.colors.Normalize(vmin=-1, vmax=1)

# Set colours for lines in middle and bottom rows
monotonic_anticlockwise_colour = cmap(norm(1.))
nonmonotonic_anticlockwise_colour = cmap(norm(.5))
nonmonotonic_clockwise_colour = cmap(norm(-.5))
monotonic_clockwise_colour = cmap(norm(-1.))

# Create legend
colours = [monotonic_anticlockwise_colour, nonmonotonic_anticlockwise_colour, monotonic_clockwise_colour, nonmonotonic_clockwise_colour]
labels = ['Monotonic Anticlockwise', 'Nonmonotonic Anticlockwise', 'Monotonic Clockwise', 'Nonmonotonic Clockwise']

patches = [mpatches.Patch(color=colours[i], label=labels[i]) for i in range(len(colours))]
ax_phase_diagram.legend(handles=patches, loc='lower right')

ax_R1 = fig.add_subplot(gs[2, 0])
ax_eta1 = fig.add_subplot(gs[2, 1])

# For the second point, want the x-scale to be the same
ax_R2 = ax_R1.twiny()
ax_eta2 = ax_eta1.twiny()

plot_coordinates(0, ax_R1, ax_eta1, plot_colours[0])
plot_coordinates(1, ax_R2, ax_eta2, plot_colours[1])

ax_R1.set_ylabel(r'$R$')
ax_R1.set_xlabel(r'$t - t_0$')
ax_eta1.set_xlabel(r'$t - t_0$')
ax_eta1.set_ylabel(r'$\eta/\pi$')

ax_R2.set_xticks([])
ax_eta2.set_xticks([])

Rmin = optimalSkyrmionRadius
ax_R1.axhline(Rmin, linestyle=':', color='gray')
ax_R1.text(0.82, 0.3, r'$R^*$', transform=ax_R1.transAxes, size=24)

plt.tight_layout(w_pad=5)

# Clean up the POV-Ray auxiliary files
os.system('rm *.png')
os.remove('Render.pov')